In [ ]:
from PIL import Image
from scipy import ndimage
import cv2
import numpy as np
import pydicom as dicom
import os
from torch.utils.data import Dataset, DataLoader
import torch
import random

def process_covidctmd(exam_path):
    image_paths = sorted(os.listdir(exam_path))
    
    exam_dict = {}
    for image_path in image_paths:
        path = os.path.join(exam_path, str(image_path).replace("b'","").replace("'", ""))
        
        ds = dicom.dcmread(path)
    
        volume = ds.pixel_array
        
        slope = 1
        intercept = -1024
        volume = (volume*slope+intercept) 

        img_min = -1250
        img_max = 250
        volume[volume<img_min] = img_min 
        volume[volume>img_max] = img_max
        volume = (volume - img_min) / (img_max - img_min)*255.0 
        volume = cv2.cvtColor(volume.astype(np.uint8), cv2.COLOR_GRAY2RGB)
    
        exam_dict[ds.SliceLocation] = volume # might sort dict if necessary
        
    return resize_volume_covidctmd(np.array(list(exam_dict.values())))

def resize_volume_covidctmd(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = img.shape[0]
    desired_width = 512
    desired_height = 512
    # Get current depth
    current_depth = img.shape[0]
    # Compute depth factor
    depth = current_depth / desired_depth
    depth_factor = 1 / depth
    # Resize across z-axis
        
    img = np.flip(img, 2)
    return np.array(img)

normal_path = '/media/riccelli/Disco 1/datasets_covid/spgc_dataset/Normal'
covid_path = '/media/riccelli/Disco 1/datasets_covid/spgc_dataset/Covid'
cp_path = '/media/riccelli/Disco 1/datasets_covid/spgc_dataset/Cap Cases'


normal_patient_paths = [
    os.path.join(os.getcwd(), normal_path, x)
    for x in os.listdir(normal_path)
]

covid_patient_paths = [
    os.path.join(os.getcwd(), covid_path, x)
    for x in os.listdir(covid_path)
]

cp_patient_paths = [
    os.path.join(os.getcwd(), cp_path, x)
    for x in os.listdir(cp_path)
]

abnormal_patient_paths = np.concatenate((np.asarray(covid_patient_paths), np.asarray(cp_patient_paths)))

patient_paths = np.concatenate((np.asarray(normal_patient_paths), np.asarray(abnormal_patient_paths)))

covidctmd_dataset = patient_paths

print("Patients with healthy lung tissue: " + str(len(normal_patient_paths)))
print("Patients with covid symptoms in lung tissue: " + str(len(covid_patient_paths)))
print("Patients with common pneumonia symptoms in lung tissue: " + str(len(cp_patient_paths)))

class_names = ['Normal', 'Covid']
idx_to_class = {i:j for i, j in enumerate(class_names)}
class_to_idx = {value:key for key,value in idx_to_class.items()}

class CustomExternalMDDataset(Dataset):
    def __init__(self, paths, transform=None, target_transform=None):
        self.paths = paths
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        exam = process_covidctmd(path)
        label = path.split('/')[-2]
        
        if label == 'Cap Cases':
            label = 'Covid'
        
        label = class_to_idx[label]
        
        if self.transform:
            exam = exam
        if self.target_transform:
            label = self.target_transform(label)
        return exam, label, path
    
covidctmd_dataset = CustomExternalMDDataset(covidctmd_dataset)
covidctmd_dataloader = DataLoader(covidctmd_dataset, batch_size=1, shuffle=False)

In [ ]:
import torchvision
from matplotlib import pyplot as plt
import numpy as np

def imshow_img(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
    
def imshow_mask(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, labels, path = next(iter(covidctmd_dataloader))
inputs = inputs.squeeze().permute(0,3,1,2)
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
print(labels)

imshow_img(out)

In [ ]:
%%time 
from get_lungs_vh_paper import get_lungs
from get_lesions_vh_paper import get_lesions
import re

plt.rcParams['figure.figsize'] = [20, 10]

j = 1

y_true = []
y_pred = []
perc_true = []
perc_pred = []
severity_true = []
severity_pred = []

for exam, label, path in iter(covidctmd_dataloader):
    exam = exam.numpy().squeeze()#.transpose(0,3,1,2)
    
    # para cada batch segmenta o pulmão
    segmented_lung_inputs, predicted_lung_inputs = get_lungs(exam, 
                                                             resize=False, 
                                                             return_masks=True,
                                                             weight=2)
    
    segmented_lesion_inputs, predicted_lesion_inputs = get_lesions(segmented_lung_inputs, 
                                                             resize=False, 
                                                             return_masks=True,
                                                             weight='Mobilenet FPN',
                                                             thresh=0.7) # >0.7
    biggest_lesion_image_idx = None
    biggest_lesion_area = 0.
    exam_percentage = []
    images_with_lesion = 0
    exam_lesion_area = 0
    for i in range(exam.shape[0]):
        
        lung_area = np.count_nonzero(predicted_lung_inputs[i,:,:,:])
        
        if lung_area > 0:
            lesion_area = np.count_nonzero(predicted_lesion_inputs[i,:,:,:])
            if lesion_area > 0:
                images_with_lesion = images_with_lesion + 1
                image_percentage = lesion_area/lung_area
                exam_percentage.append(image_percentage)
                exam_lesion_area = exam_lesion_area + lesion_area
            
                if lesion_area > biggest_lesion_area:
                    biggest_lesion_image_idx = i
                    biggest_lesion_area = lesion_area
    
    if len(exam_percentage) > 0:
        cal_exam_percentage = np.mean(exam_percentage)
    else:
        cal_exam_percentage = 0
        
    if images_with_lesion > 3 and exam_lesion_area > 1500:
        pred_label = 1
    else:
        pred_label = 0

        
    print(f'Exam path {path[0].split("/")[-2:]}')
    print(f'Exam {j} label: {label.item()}, Predicted label: {pred_label}')
    print(f'Images with lesion {images_with_lesion} and Biggest Lesion area {biggest_lesion_area} and Exam lesion area {exam_lesion_area}')
    
    if biggest_lesion_image_idx != None:
        plt.subplot(141)
        plt.axis('off')
        plt.imshow(exam[biggest_lesion_image_idx,:,:,:])
        plt.subplot(142)
        plt.axis('off')
        plt.imshow(segmented_lung_inputs[biggest_lesion_image_idx,:,:,:])
        plt.subplot(143)
        plt.axis('off')
        plt.imshow(predicted_lesion_inputs[biggest_lesion_image_idx,:,:,:])
        
        plt.subplot(144)
        img = (predicted_lesion_inputs[biggest_lesion_image_idx,:,:,:][:,:,0]).astype(np.uint8)
        contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        painted_img = cv2.drawContours(exam[biggest_lesion_image_idx,:,:,:].copy(), contours, -1, (255,0,0), 2)
        img2 = (predicted_lung_inputs[biggest_lesion_image_idx,:,:,:][:,:,0]).astype(np.uint8)
        contours, hierarchy = cv2.findContours(img2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        painted_img = cv2.drawContours(painted_img, contours, -1, (0,255,0), 2)
        plt.axis('off')
        plt.imshow(painted_img)
        plt.show()
        
    j = j+1
    
    
    
    perc_pred.append(cal_exam_percentage)    
    y_true.append(label.item())
    y_pred.append(pred_label)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay, mean_squared_error, mean_absolute_error

acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
rec = recall_score(y_true, y_pred)
spec = recall_score(y_true, y_pred, pos_label=0)
print(f"MobilenetV2 FPN & {round(acc*100,2)} & {round(f1*100,2)} & {round(prec*100,2)} & {round(rec*100,2)} & {round(spec*100,2)} \\\\")  

confusion_matrix = confusion_matrix(y_true, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ['Normal', 'Covid'])
cm_display.plot(cmap="Blues")
plt.show()